# Ch17: Encapsulation (封裝) - 詳解範例

本檔案提供 5 個詳細的封裝實作範例，每個範例都包含完整的問題分析、設計思路與實作步驟。

---

## 範例 1: Temperature 類別 - Celsius/Fahrenheit 驗證

### 📋 問題描述
設計一個 `Temperature` 類別，需要：
1. 儲存溫度值（以攝氏為主）
2. 限制溫度不得低於絕對零度（-273.15°C）
3. 提供攝氏與華氏的轉換屬性
4. 使用封裝保護內部資料

### 💡 設計思路
- 使用私有屬性 `_celsius` 儲存實際溫度
- 使用 `@property` 提供 `celsius` 和 `fahrenheit` 屬性
- 在 setter 中加入驗證邏輯
- 提供溫度轉換方法

### 🔧 實作步驟

In [ ]:
class Temperature:
    """溫度類別 - 支援攝氏/華氏轉換與驗證"""
    
    ABSOLUTE_ZERO = -273.15  # 絕對零度（攝氏）
    
    def __init__(self, celsius=0):
        """初始化溫度（預設 0°C）"""
        self._celsius = 0
        self.celsius = celsius  # 使用 setter 驗證
    
    @property
    def celsius(self):
        """取得攝氏溫度"""
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        """設定攝氏溫度（驗證是否低於絕對零度）"""
        if value < self.ABSOLUTE_ZERO:
            raise ValueError(f"溫度不能低於絕對零度 ({self.ABSOLUTE_ZERO}°C)")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        """取得華氏溫度（計算屬性）"""
        return self._celsius * 9/5 + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value):
        """設定華氏溫度（自動轉換為攝氏）"""
        celsius = (value - 32) * 5/9
        self.celsius = celsius  # 透過 celsius setter 驗證
    
    def __str__(self):
        return f"{self._celsius:.2f}°C ({self.fahrenheit:.2f}°F)"


# 測試範例
print("=== Temperature 類別測試 ===")

# 建立溫度物件
temp = Temperature(25)
print(f"初始溫度: {temp}")

# 修改攝氏溫度
temp.celsius = 100
print(f"設定為 100°C: {temp}")

# 修改華氏溫度
temp.fahrenheit = 32
print(f"設定為 32°F: {temp}")

# 測試絕對零度邊界
temp.celsius = -273.15
print(f"絕對零度: {temp}")

# 測試錯誤處理
try:
    temp.celsius = -300
except ValueError as e:
    print(f"錯誤: {e}")

### 📊 執行結果說明
- ✅ 成功驗證溫度下限
- ✅ 自動進行攝氏/華氏轉換
- ✅ 使用 `@property` 提供簡潔的介面
- ✅ 私有屬性 `_celsius` 受保護

---

## 範例 2: User Account - 加密密碼

### 📋 問題描述
設計一個 `UserAccount` 類別，需要：
1. 儲存使用者名稱和密碼
2. 密碼不應該以明文儲存
3. 提供密碼驗證功能
4. 密碼長度至少 8 個字元

### 💡 設計思路
- 使用雙底線 `__password` 進行名稱修飾（Name Mangling）
- 使用 `hashlib` 模組進行密碼雜湊
- 只提供密碼驗證方法，不提供密碼讀取
- 在設定密碼時驗證長度

### 🔧 實作步驟

In [ ]:
import hashlib

class UserAccount:
    """使用者帳戶類別 - 加密密碼儲存"""
    
    MIN_PASSWORD_LENGTH = 8
    
    def __init__(self, username, password):
        """初始化使用者帳戶"""
        self.username = username
        self.__password_hash = None
        self.set_password(password)
    
    def __hash_password(self, password):
        """私有方法：密碼雜湊（使用 SHA-256）"""
        return hashlib.sha256(password.encode()).hexdigest()
    
    def set_password(self, password):
        """設定密碼（加密儲存）"""
        # 驗證密碼長度
        if len(password) < self.MIN_PASSWORD_LENGTH:
            raise ValueError(f"密碼長度至少需要 {self.MIN_PASSWORD_LENGTH} 個字元")
        
        # 儲存雜湊值
        self.__password_hash = self.__hash_password(password)
    
    def verify_password(self, password):
        """驗證密碼是否正確"""
        return self.__hash_password(password) == self.__password_hash
    
    def __str__(self):
        return f"UserAccount(username='{self.username}', password='***')"


# 測試範例
print("=== UserAccount 類別測試 ===")

# 建立使用者帳戶
user = UserAccount("alice", "mypassword123")
print(f"建立帳戶: {user}")

# 驗證正確密碼
print(f"\n驗證 'mypassword123': {user.verify_password('mypassword123')}")

# 驗證錯誤密碼
print(f"驗證 'wrongpassword': {user.verify_password('wrongpassword')}")

# 修改密碼
user.set_password("newpassword456")
print(f"\n密碼已更新")
print(f"驗證新密碼: {user.verify_password('newpassword456')}")
print(f"驗證舊密碼: {user.verify_password('mypassword123')}")

# 測試密碼長度驗證
try:
    user.set_password("short")
except ValueError as e:
    print(f"\n錯誤: {e}")

# 嘗試直接存取私有屬性（會失敗）
print(f"\n嘗試存取 __password_hash:")
try:
    print(user.__password_hash)
except AttributeError as e:
    print(f"無法存取: {e}")

# Name Mangling 後的實際屬性名稱
print(f"\n透過 Name Mangling 存取（不建議）:")
print(f"實際雜湊值: {user._UserAccount__password_hash[:20]}...")

### 📊 執行結果說明
- ✅ 密碼以雜湊值儲存，無法還原
- ✅ 使用雙底線進行名稱修飾，提高安全性
- ✅ 密碼長度驗證正常運作
- ✅ 只能驗證密碼，無法直接讀取

---

## 範例 3: Rectangle - 計算屬性 area

### 📋 問題描述
設計一個 `Rectangle` 類別，需要：
1. 儲存寬度和高度
2. 寬度和高度必須為正數
3. 提供計算面積的屬性
4. 提供計算周長的屬性

### 💡 設計思路
- 使用私有屬性儲存寬高
- 使用 `@property` 提供驗證的 getter/setter
- `area` 和 `perimeter` 為唯讀計算屬性
- 在 setter 中驗證數值必須為正

### 🔧 實作步驟

In [ ]:
class Rectangle:
    """矩形類別 - 計算屬性示範"""
    
    def __init__(self, width, height):
        """初始化矩形"""
        self._width = 0
        self._height = 0
        self.width = width    # 使用 setter 驗證
        self.height = height  # 使用 setter 驗證
    
    @property
    def width(self):
        """取得寬度"""
        return self._width
    
    @width.setter
    def width(self, value):
        """設定寬度（必須為正數）"""
        if value <= 0:
            raise ValueError("寬度必須為正數")
        self._width = value
    
    @property
    def height(self):
        """取得高度"""
        return self._height
    
    @height.setter
    def height(self, value):
        """設定高度（必須為正數）"""
        if value <= 0:
            raise ValueError("高度必須為正數")
        self._height = value
    
    @property
    def area(self):
        """計算面積（唯讀屬性）"""
        return self._width * self._height
    
    @property
    def perimeter(self):
        """計算周長（唯讀屬性）"""
        return 2 * (self._width + self._height)
    
    def scale(self, factor):
        """等比例縮放"""
        if factor <= 0:
            raise ValueError("縮放比例必須為正數")
        self._width *= factor
        self._height *= factor
    
    def __str__(self):
        return f"Rectangle({self._width} × {self._height})"
    
    def __repr__(self):
        return f"Rectangle(width={self._width}, height={self._height})"


# 測試範例
print("=== Rectangle 類別測試 ===")

# 建立矩形
rect = Rectangle(5, 3)
print(f"建立矩形: {rect}")
print(f"面積: {rect.area}")
print(f"周長: {rect.perimeter}")

# 修改尺寸
rect.width = 10
rect.height = 6
print(f"\n修改尺寸後: {rect}")
print(f"面積: {rect.area}")
print(f"周長: {rect.perimeter}")

# 等比例縮放
rect.scale(0.5)
print(f"\n縮放 0.5 倍後: {rect}")
print(f"面積: {rect.area}")

# 測試驗證
print("\n=== 驗證測試 ===")
try:
    rect.width = -5
except ValueError as e:
    print(f"寬度驗證錯誤: {e}")

try:
    rect.height = 0
except ValueError as e:
    print(f"高度驗證錯誤: {e}")

# 測試唯讀屬性
try:
    rect.area = 100
except AttributeError as e:
    print(f"面積為唯讀屬性: can't set attribute")

### 📊 執行結果說明
- ✅ 計算屬性自動更新
- ✅ 唯讀屬性無法被修改
- ✅ 數值驗證正常運作
- ✅ 封裝內部狀態，提供簡潔介面

---

## 範例 4: Product - 價格驗證

### 📋 問題描述
設計一個 `Product` 類別，需要：
1. 儲存商品名稱、價格、折扣
2. 價格必須為非負數
3. 折扣範圍 0-100（百分比）
4. 提供計算折扣後價格的屬性

### 💡 設計思路
- 使用 `@property` 封裝價格和折扣
- 在 setter 中加入範圍驗證
- `final_price` 為計算屬性
- 提供格式化的價格顯示

### 🔧 實作步驟

In [ ]:
class Product:
    """商品類別 - 價格驗證示範"""
    
    def __init__(self, name, price, discount=0):
        """初始化商品
        
        Args:
            name: 商品名稱
            price: 原價
            discount: 折扣百分比 (0-100)
        """
        self.name = name
        self._price = 0
        self._discount = 0
        self.price = price      # 使用 setter 驗證
        self.discount = discount  # 使用 setter 驗證
    
    @property
    def price(self):
        """取得原價"""
        return self._price
    
    @price.setter
    def price(self, value):
        """設定原價（必須為非負數）"""
        if value < 0:
            raise ValueError("價格不能為負數")
        self._price = value
    
    @property
    def discount(self):
        """取得折扣百分比"""
        return self._discount
    
    @discount.setter
    def discount(self, value):
        """設定折扣（範圍 0-100）"""
        if not 0 <= value <= 100:
            raise ValueError("折扣必須在 0-100 之間")
        self._discount = value
    
    @property
    def final_price(self):
        """計算折扣後價格（唯讀）"""
        return self._price * (1 - self._discount / 100)
    
    @property
    def discount_amount(self):
        """計算折扣金額（唯讀）"""
        return self._price - self.final_price
    
    def apply_extra_discount(self, extra_percent):
        """套用額外折扣"""
        new_discount = min(self._discount + extra_percent, 100)
        self.discount = new_discount
    
    def __str__(self):
        if self._discount > 0:
            return (f"{self.name}: ${self._price:.2f} "
                   f"({self._discount}% off) → ${self.final_price:.2f}")
        return f"{self.name}: ${self._price:.2f}"


# 測試範例
print("=== Product 類別測試 ===")

# 建立商品（無折扣）
laptop = Product("Laptop", 1000)
print(f"商品 1: {laptop}")

# 建立商品（有折扣）
phone = Product("Smartphone", 800, 20)
print(f"商品 2: {phone}")
print(f"  折扣金額: ${phone.discount_amount:.2f}")

# 修改價格和折扣
print("\n=== 修改價格與折扣 ===")
phone.price = 900
phone.discount = 30
print(f"更新後: {phone}")
print(f"  折扣金額: ${phone.discount_amount:.2f}")

# 套用額外折扣
print("\n=== 套用額外折扣 ===")
phone.apply_extra_discount(10)  # 再打 9 折
print(f"額外 10% 折扣後: {phone}")

# 驗證測試
print("\n=== 驗證測試 ===")
try:
    laptop.price = -100
except ValueError as e:
    print(f"價格驗證: {e}")

try:
    laptop.discount = 150
except ValueError as e:
    print(f"折扣驗證: {e}")

# 測試唯讀屬性
try:
    phone.final_price = 500
except AttributeError:
    print(f"final_price 為唯讀屬性")

### 📊 執行結果說明
- ✅ 價格和折扣驗證正常
- ✅ 自動計算折扣後價格
- ✅ 提供額外折扣功能
- ✅ 唯讀屬性保護計算結果

---

## 範例 5: Counter - 私有狀態管理

### 📋 問題描述
設計一個 `Counter` 類別，需要：
1. 提供遞增、遞減功能
2. 設定計數上下限
3. 記錄操作歷史
4. 提供重置功能

### 💡 設計思路
- 使用私有屬性儲存計數值、上下限、歷史
- 使用 `@property` 提供唯讀的狀態查詢
- 在修改方法中加入邊界檢查
- 記錄每次操作到歷史列表

### 🔧 實作步驟

In [ ]:
from datetime import datetime

class Counter:
    """計數器類別 - 私有狀態管理示範"""
    
    def __init__(self, initial=0, min_value=None, max_value=None):
        """初始化計數器
        
        Args:
            initial: 初始值
            min_value: 最小值限制（None 表示無限制）
            max_value: 最大值限制（None 表示無限制）
        """
        self._value = initial
        self._initial = initial
        self._min = min_value
        self._max = max_value
        self._history = []
        self._log_operation("initialized", initial)
    
    def _log_operation(self, operation, value):
        """私有方法：記錄操作歷史"""
        timestamp = datetime.now().strftime("%H:%M:%S")
        self._history.append({
            'time': timestamp,
            'operation': operation,
            'value': value
        })
    
    @property
    def value(self):
        """取得當前計數值（唯讀）"""
        return self._value
    
    @property
    def history(self):
        """取得操作歷史（唯讀，返回副本）"""
        return self._history.copy()
    
    def increment(self, step=1):
        """遞增計數"""
        new_value = self._value + step
        
        # 檢查上限
        if self._max is not None and new_value > self._max:
            raise ValueError(f"計數值不能超過 {self._max}")
        
        self._value = new_value
        self._log_operation(f"increment(+{step})", self._value)
        return self._value
    
    def decrement(self, step=1):
        """遞減計數"""
        new_value = self._value - step
        
        # 檢查下限
        if self._min is not None and new_value < self._min:
            raise ValueError(f"計數值不能低於 {self._min}")
        
        self._value = new_value
        self._log_operation(f"decrement(-{step})", self._value)
        return self._value
    
    def reset(self):
        """重置為初始值"""
        self._value = self._initial
        self._log_operation("reset", self._value)
        return self._value
    
    def clear_history(self):
        """清除歷史記錄"""
        self._history.clear()
        self._log_operation("history_cleared", self._value)
    
    def print_history(self):
        """顯示操作歷史"""
        print("\n=== 操作歷史 ===")
        for record in self._history:
            print(f"[{record['time']}] {record['operation']:20s} → {record['value']}")
    
    def __str__(self):
        return f"Counter(value={self._value}, min={self._min}, max={self._max})"


# 測試範例
print("=== Counter 類別測試 ===")

# 建立計數器（限制範圍 0-10）
counter = Counter(initial=5, min_value=0, max_value=10)
print(f"建立計數器: {counter}")

# 遞增操作
counter.increment()
counter.increment(2)
print(f"遞增後: value = {counter.value}")

# 遞減操作
counter.decrement()
counter.decrement(3)
print(f"遞減後: value = {counter.value}")

# 重置
counter.reset()
print(f"重置後: value = {counter.value}")

# 顯示歷史
counter.print_history()

# 測試邊界
print("\n=== 邊界測試 ===")
try:
    for _ in range(6):
        counter.increment()
        print(f"當前值: {counter.value}")
except ValueError as e:
    print(f"上限錯誤: {e}")

counter.reset()
try:
    for _ in range(6):
        counter.decrement()
        print(f"當前值: {counter.value}")
except ValueError as e:
    print(f"下限錯誤: {e}")

# 測試唯讀屬性
print("\n=== 唯讀屬性測試 ===")
try:
    counter.value = 100
except AttributeError:
    print("value 為唯讀屬性，無法直接修改")

# 歷史記錄是副本，修改不影響原始資料
history_copy = counter.history
history_copy.append({'fake': 'data'})
print(f"原始歷史長度: {len(counter.history)}")
print(f"副本歷史長度: {len(history_copy)}")

### 📊 執行結果說明
- ✅ 私有屬性保護內部狀態
- ✅ 邊界檢查正常運作
- ✅ 操作歷史完整記錄
- ✅ 唯讀屬性返回資料副本，防止外部修改

---

## 📚 本章重點回顧

### 封裝的核心概念
1. **私有屬性保護**：使用 `_` 或 `__` 前綴
2. **@property 裝飾器**：提供 getter/setter 控制
3. **計算屬性**：基於其他屬性動態計算
4. **唯讀屬性**：只提供 getter，沒有 setter
5. **驗證邏輯**：在 setter 中加入資料驗證

### 實作模式總結

| 模式 | 使用時機 | 範例 |
|:-----|:---------|:-----|
| 私有屬性 + Property | 需要驗證或計算的屬性 | Temperature, Product |
| 計算屬性（唯讀） | 基於其他屬性計算的值 | Rectangle.area, Product.final_price |
| Name Mangling | 需要高度保護的敏感資料 | UserAccount.__password_hash |
| 返回副本 | 防止外部修改內部集合 | Counter.history |
| 私有方法 | 內部使用的輔助方法 | Counter._log_operation |

### 最佳實踐
1. ✅ 預設使用 `_` 單底線表示私有（約定）
2. ✅ 敏感資料使用 `__` 雙底線（Name Mangling）
3. ✅ 在 setter 中加入驗證，確保資料有效性
4. ✅ 計算屬性設為唯讀，避免不一致
5. ✅ 返回可變物件時使用 `.copy()` 保護內部狀態

---

**繼續練習**: 請前往 `03-practice.ipynb` 進行課堂練習！